In [1]:
%reset -f
import matplotlib.pyplot as plt
import os
import numpy as np
import parsing
import copy
import vertex
import time
import movement
import cell
import geometry
import data_holder
import math
import h5py
import simulate
import csv
import pandas as pd
import plotting
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

In [ ]:
simulate.run_active_vertex_model(
    "vertices.csv", "cell_indices.csv", "output1.hdf5", 6, 6, 1, 1, 0.01, max_time=1
)

In [ ]:
with open("vertices.csv", "r") as f:
    reader = csv.reader(f)
    data = list(reader)

data_array = np.array(data, dtype=float)

with open("cell_indices.csv", "r") as f:
    reader = csv.reader(f)
    data = list(reader)

cell_array = np.array(data, dtype=int)

d = data_holder.Data_Holder(data_array, cell_array, lx=6, ly=6, KA=1, KP=1, dt=0.01)

In [ ]:
attrs = ["cell_list", "vert_adjcent_cells", "vert_list"]
[attr for attr in dir(d) if not attr.startswith("_") and attr not in attrs]

In [ ]:
d.__getattribute__("KA")

In [ ]:
import h5py
import numpy as np

# Example data with lists of different lengths containing floats
data = [[0.1, 0.2], [0.3, 0.4, 0.5, 0.6], [0.7, 0.8]]

# Use h5py special dtype for variable-length sequences of floats
vlen_dtype = h5py.special_dtype(vlen=np.dtype("float32"))

# Convert the data to an array of object type
# data_array = np.array(data, dtype=object)

# Write to HDF5 file
with h5py.File("output.hdf5", "w") as f:
    # Create the dataset with variable-length data type
    dset = f.create_dataset("step/verticesss", data=data)

In [ ]:
import h5py
import numpy as np

# Example data with lists of different lengths containing floats
data = [[0.1, 0.2], [0.3, 0.4, 0.5, 0.6], [0.7, 0.8]]

# Convert data to a NumPy array of dtype=object
data_array = np.array(data, dtype=object)

# Define a variable-length float dtype
vlen_dtype = h5py.special_dtype(vlen=np.dtype("float64"))

# Write to HDF5 file
with h5py.File("output.hdf5", "w") as f:
    # Create the dataset with variable-length data type
    dset = f.create_dataset("step/verticess", data=data, dtype=vlen_dtype)
    dset.attrs["bruh"] = 63

# Read the data back from the file
with h5py.File("output.hdf5", "r") as f:
    dset = f["step/verticess"]
    print(dset[:])  # Print the stored data
    print(dset.attrs["bruh"])  # Print the attribute

In [ ]:
_dtype(vlen=np.dtype("float64"))
data_array = np.array(l, dtype=object)
with h5py.File("output.hdf5", "w") as f:
    hello = f.create_dataset(f"step/verticess", data=data_array, dtype=vlen_dtype)

In [ ]:
with h5py.File("output.hdf5", "r") as f:
    dset = f["step/verticess"]
    print(dset.attrs["bruh"])

In [ ]:
data.__getattribute__()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 6)
ax.set_ylim(0, 6)
with h5py.File("output1.hdf5", "r") as f:
    dset = f["step_00000/cell_vertices"]
    j = f["step_00000/vertices"]
    lx = f["step_00000/vertices"].attrs["lx"]
    ly = f["step_00000/vertices"].attrs["ly"]
    graph_lines(dset, ax, lx, ly)
    ax.scatter(j[:, 0], j[:, 1])

In [ ]:
def graph_lines(cells, ax, lx, ly, color="black"):
    # graph lines
    for cell in cells:
        for i in range(len(cell)):
            x1 = cell[i][0]
            y1 = cell[i][1]
            x2 = cell[(i + 1) % len(cell)][0]
            y2 = cell[(i + 1) % len(cell)][1]
            # if any boundaries broken, must check which boundaries
            # then draw 2 segments connecting to the edge of the border
            if abs(x1 - x2) > lx / 2 or abs(y1 - y2) > ly / 2:
                # segment 1
                if abs(x1 - x2) > lx / 2:
                    if x1 > x2:
                        x2 += lx
                    else:
                        x1 += lx
                if abs(y1 - y2) > ly / 2:
                    if y1 > y2:
                        y2 += ly
                    else:
                        y1 += ly
                ax.plot([x1, x2], [y1, y2], color=color)
                # segment 2
                x1 = cell[i][0]
                y1 = cell[i][1]
                x2 = cell[(i + 1) % len(cell)][0]
                y2 = cell[(i + 1) % len(cell)][1]
                if abs(x1 - x2) > lx / 2:
                    if x1 > x2:
                        x1 -= lx
                    else:
                        x2 -= lx
                if abs(y1 - y2) > ly / 2:
                    if y1 > y2:
                        y1 -= ly
                    else:
                        y2 -= ly
                ax.plot([x1, x2], [y1, y2], color=color)
            # no boundaries broken
            # when no periodic boundaries broken, just draw one segment
            else:
                ax.plot([x1, x2], [y1, y2], color=color)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np


def plot_data(data):

    fig, ax = plt.subplots()
    # fig = plt.figure(figsize = (6,4))

    # Set limits for x and y axis
    ax.set_xlim(0, 6)
    ax.set_ylim(0, 6)
    # ax.set_xticks([])
    # ax.set_yticks([])

    scat = ax.scatter([], [], c="red")

    def init():
        positions = np.array([[v[0], v[1]] for v in data.vert_list])
        graph_lines(data, ax)
        scat.set_offsets(positions)  #
        return (scat,)

    def animate(i):
        if i == 0:
            print("hi")
            movement.move_test(data.vert_list, data)
        # if i > 9.91 and i < 10.1:
        #     print (i)
        #     movement.move_test(data.vert_list, data)
        # spvert.append([float(data.v[0]), float(data.v[1])])
        # spvert_plus.append([float((data.v[0] - 2.6) % data.lx), float(data.v[1])])
        movement.move_vertices(data.vert_list, data)
        positions = np.array([[v[0], v[1]] for v in data.vert_list])  #
        # jank solution to clearing lines
        ax.clear()
        ax.set_xlim(0, 6)
        ax.set_ylim(0, 6)
        scat = ax.scatter([], [], c="red")
        # ax.set_xticks([])
        # ax.set_yticks([])
        graph_lines(data, ax)
        scat.set_offsets(positions)  #
        return (scat,)

    plt.close()
    anim = animation.FuncAnimation(
        fig,
        animate,
        init_func=init,
        frames=np.arange(0, 5, 0.1),
        interval=100,
        blit=True,
    )
    return anim


# anim = plot_data(data)
# #anim.save("vertex_model_beta.mp4", writer = "ffmpeg")
# from IPython.display import HTML
# HTML(anim.to_jshtml())

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 6)
ax.set_ylim(0, 6)
with h5py.File("output1.hdf5", "r") as f:
    dset = f["step_00000/edges"]
    j = f["step_00000/vertices"]
    lx = f["step_00000/vertices"].attrs["lx"]
    ly = f["step_00000/vertices"].attrs["ly"]
    plotting.graph_frame(ax, j, dset, lx, ly)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np


def plot_data(data):

    fig, ax = plt.subplots()
    # fig = plt.figure(figsize = (6,4))

    # Set limits for x and y axis
    ax.set_xlim(0, 6)
    ax.set_ylim(0, 6)
    # ax.set_xticks([])
    # ax.set_yticks([])

    scat = ax.scatter([], [], c="red")

    def init():
        # positions = np.array([[v[0], v[1]] for v in data.vert_list])
        # graph_lines(data, ax)
        # scat.set_offsets(positions) #
        plotting.graph_frame()
        return (scat,)

    def animate(i):
        movement.move_vertices(data.vert_list, data)
        # positions = np.array([[v[0], v[1]] for v in data.vert_list]) #
        # # jank solution to clearing lines
        # ax.clear()
        # ax.set_xlim(0, 6)
        # ax.set_ylim(0, 6)
        # scat = ax.scatter([], [], c="red")
        # # ax.set_xticks([])
        # # ax.set_yticks([])
        # graph_lines(data, ax)
        # scat.set_offsets(positions) #
        ax.clear()
        ax.set_xlim(0, 6)
        ax.set_ylim(0, 6)
        return (scat,)

    plt.close()
    anim = animation.FuncAnimation(
        fig,
        animate,
        init_func=init,
        frames=np.arange(0, 20, 0.1),
        interval=100,
        blit=True,
    )
    return anim


# anim = plot_data(data)
# #anim.save("vertex_model_beta.mp4", writer = "ffmpeg")
# from IPython.display import HTML
# HTML(anim.to_jshtml())

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np

plt.rcParams["animation.embed_limit"] = 40  # Setting the limit to 30 MB


def plot_data(data):

    fig, ax = plt.subplots()
    # fig = plt.figure(figsize = (6,4))

    # Set limits for x and y axis
    ax.set_xlim(0, 6)
    ax.set_ylim(0, 6)
    ax.set_xticks([])
    ax.set_yticks([])

    # scat = ax.scatter([], [], c = "red")

    # file = None

    # count = 0

    def init():
        positions = np.array([[v[0], v[1]] for v in data.vert_list])
        graph_lines(data, ax)
        scat.set_offsets(positions)  #
        # energy.append(movement.calc_energy(data)) # check energy
        # print(data.A0, data.cell_list[0].area, data.P0, data.cell_list[0].area)
        return (scat,)

    def animate(i):
        print(i)
        # if i == 0:
        #     print("hi")
        #     movement.move_test(data.vert_list, data)
        # if i > 9.91 and i < 10.1:
        #     print (i)
        #     movement.move_test(data.vert_list, data)
        # spvert.append([float(data.v[0]), float(data.v[1])])
        # spvert_plus.append([float((data.v[0] - 2.6) % data.lx), float(data.v[1])])
        movement.move_vertices(data.vert_list, data)
        movement.t1_transition_check(data)
        positions = np.array([[v[0], v[1]] for v in data.vert_list])  #
        # with h5py.File("output.hdf5", "a") as f:
        #     s = f"step_{int(round(i, 1) * 10)}"
        #     if s in f:
        #         del f[s]
        #     f.create_dataset(s, data = positions)
        # jank solution to clearing lines
        ax.clear()
        ax.set_xlim(0, 6)
        ax.set_ylim(0, 6)
        scat = ax.scatter([], [], c="red")
        ax.set_xticks([])
        ax.set_yticks([])
        graph_lines(data, ax)
        scat.set_offsets(positions)  #
        # energy.append(movement.calc_energy(data)) # check energy
        return (scat,)

    plt.close()
    anim = animation.FuncAnimation(
        fig,
        animate,
        init_func=init,
        frames=np.arange(0, 20, 0.1),
        interval=100,
        blit=True,
    )
    return anim


anim = plot_data(data)
# anim.save("vertex_model_working_noise.mp4", writer = "ffmpeg")
from IPython.display import HTML

HTML(anim.to_jshtml())

In [ ]:
import h5py
import numpy as np

# Example data with lists of different lengths containing floats
cell_vertices = [
    np.array([0.1, 0.2], dtype="float64"),
    np.array([0.3, 0.4, 0.5, 0.6], dtype="float64"),
    np.array([0.7, 0.8], dtype="float64"),
]

# Convert list of NumPy arrays to a NumPy array of dtype=object
data_array = np.array(cell_vertices, dtype=object)

# Define a variable-length float dtype
vlen_dtype = h5py.special_dtype(vlen=np.dtype("float64"))

# Write to HDF5 file
with h5py.File("output.hdf5", "w") as f:
    # Create the dataset with variable-length float data
    dset = f.create_dataset("step/verticess", data=data_array, dtype=vlen_dtype)
    dset.attrs["bruh"] = 63

# Read the data back from the file
with h5py.File("output.hdf5", "r") as f:
    dset = f["step/verticess"]
    print(dset[:])  # Print the stored data
    print(dset.attrs["bruh"])  # Print the attribute

In [ ]:
single_cell_vertices_1 = [[3, 4], [5, 6], [7, 8]]  # 2
single_cell_vertices_2 = [[3, 4], [5, 6], [7, 8], [9, 10]]  # 2
np.array(single_cell_vertices_1)
np.array(single_cell_vertices_2)
with h5py.File("pain.hdf5", "w") as f:
    dset = f.create_dataset(
        "death",
        (len(cell_vertices),),
        dtype=h5py.vlen_dtype(np.dtype("float64")),
        chunks=True,
        compression="gzip",
    )

In [ ]:
import h5py
import numpy as np
import os


def save_simulation_step(data, output_filepath, step, curr_time, attrs_to_save):
    # Move vertices and save other steps
    while curr_time < data.max_time:
        print(curr_time)
        step += 1
        curr_time += data.dt

        # Move vertices and check transitions
        movement.move_vertices(data.vert_list, data)
        movement.t1_transition_check(data)

        # Prepare positions array
        positions = np.array([[v[0], v[1]] for v in data.vert_list])

        try:
            # Open HDF5 file
            with h5py.File(output_filepath, "a") as f:
                # Save vertices
                path = os.path.join(f"step_{step:05d}", "vertices")
                if path in f:
                    del f[path]
                verts = f.create_dataset(
                    path, data=positions, chunks=True, compression="gzip"
                )

                # Prepare cell_vertices
                cell_vertices = [
                    np.array(
                        [[v[0], v[1]] for v in cell.vert_obj_list], dtype="float64"
                    )
                    for cell in data.cell_list
                ]

                # Save cell_vertices as a variable-length dataset
                path = os.path.join(f"step_{step:05d}", "cell_vertices")
                if path in f:
                    del f[path]
                dset = f.create_dataset(
                    path,
                    (len(cell_vertices),),
                    dtype=h5py.vlen_dtype(np.dtype("float64")),
                    chunks=True,
                    compression="gzip",
                )
                dset[:] = cell_vertices

                # Save other attributes
                for attr in attrs_to_save:
                    if hasattr(data, attr):
                        verts.attrs[attr] = getattr(data, attr)
                verts.attrs["energy"] = movement.calc_energy(data)
                verts.attrs["step"] = step
                verts.attrs["curr_time"] = curr_time
        except Exception as e:
            print(f"Error saving step {step}: {e}")
            break  # Exit the loop if there is an error to prevent crashing the kernel

In [ ]:
# read out cells
for i in range(1):
    with h5py.File("output1.hdf5", "r") as f:
        path = os.path.join(f"step_00023", "cell_vertices")
        if path in f:
            cell_vertices = []
            dset = f[path]
            for i in range(len(dset)):
                cell_vertices.append(dset[i].reshape(-1, 2))
        for cell in cell_vertices:
            print(len(cell))

In [ ]:
# read out cells
for i in range(1000):
    with h5py.File("output1.hdf5", "r") as f:
        path = os.path.join(f"step_{i:05d}", "cell_vertices")
        if path in f:
            cell_vertices = []
            dset = f[path]
            for i in range(len(dset)):
                cell_vertices.append(dset[i].reshape(-1, 2))
        for cell in cell_vertices:
            if len(cell) != 6:
                print(cell)
                print(i)

In [ ]:
simulate.run_active_vertex_model(
    "vertices.csv", "cell_indices.csv", "output1.hdf5", 6, 6, 1, 1, 0.01, max_time=10
)

In [ ]:
with h5py.File("output1.hdf5", "r") as f:
    dset = f["step_00000/edges"]
    print(dset[0])

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np

plt.rcParams["animation.embed_limit"] = 40  # Setting the limit to 30 MB

path = "output1.hdf5"

anim = plot_data(path)
# anim.save("vertex_model_working_noise.mp4", writer = "ffmpeg")
from IPython.display import HTML

HTML(anim.to_jshtml())

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import h5py
import plotting  # Ensure this is your custom plotting module

plt.rcParams["animation.embed_limit"] = 40  # Setting the limit to 40 MB

path = "output1.hdf5"


def plot_data(path, stepsize=10):

    try:
        with h5py.File(path, "r") as f:
            totalframes = len(f.keys())
    except FileNotFoundError:
        print("No File Found")
        return

    fig, ax = plt.subplots()

    # Set limits for x and y axis
    ax.set_xlim(0, 6)
    ax.set_ylim(0, 6)
    ax.set_xticks([])
    ax.set_yticks([])

    scat = ax.scatter([], [], c="red")

    def init():
        with h5py.File(path, "r") as f:
            dset = f["step_00000/edges"]
            j = f["step_00000/vertices"]
            lx = f["step_00000/vertices"].attrs["lx"]
            ly = f["step_00000/vertices"].attrs["ly"]
            plotting.graph_frame(ax, j, dset, lx, ly)
        return (scat,)

    def animate(i):
        ax.clear()
        ax.set_xlim(0, 6)
        ax.set_ylim(0, 6)
        ax.set_xticks([])
        ax.set_yticks([])
        with h5py.File(path, "r") as f:
            dset = f[f"step_{i:05d}/edges"]
            j = f[f"step_{i:05d}/vertices"]
            lx = f[f"step_{i:05d}/vertices"].attrs["lx"]
            ly = f[f"step_{i:05d}/vertices"].attrs["ly"]
            plotting.graph_frame(ax, j, dset, lx, ly)
        return (scat,)

    plt.close()
    anim = animation.FuncAnimation(
        fig,
        animate,
        init_func=init,
        frames=np.arange(0, totalframes, stepsize),
        interval=100,
        blit=True,
    )
    return anim


anim = plot_data(path)
# anim.save("vertex_model_working_noise.mp4", writer="ffmpeg")
from IPython.display import HTML

HTML(anim.to_jshtml())

In [ ]:
def __init__(self, vert_list, cell_list, KA = 1, KP = 1, \
                 dt = 0.01, D = 2, v0 = 0.01, max_time = 2, p0 = 1):
    
data_holder.Data_Holder(
    
    KA=1,
    KP=1,
    dt=0.01,
    min_d=0.03,
    D=2,
    v0=0.05,
    max_time=5,)


In [ ]:
# Plot energy
energy = []
count = []
with h5py.File("output1.hdf5", "r") as f:
    for i in range(len(f.keys())):
        count.append(i)
        energy.append(f[f"step_{i:05d}/vertices"].attrs["energy"])

d = {"count": count, "energy": energy}

l = pd.DataFrame.from_dict(d)
l.plot.scatter(x="count", y="energy")
avg = sum(energy) / len(energy)
plt.plot([count[0], count[-1]], [avg, avg], "r--")
plt.show()
print(avg)
plt.close()
plt.close()
plt.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the seed for reproducibility (optional)
np.random.seed(4)

# Generate 100 random vertices within a 10x10 box
num_vertices = 50
x_coords = np.random.uniform(0, 5, num_vertices)
y_coords = np.random.uniform(0, 5, num_vertices)

# Combine x and y coordinates into a list of vertices with labels
vertices = {i: (x_coords[i], y_coords[i]) for i in range(num_vertices)}

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))

# Scatter plot of vertices
x_vals, y_vals = zip(*vertices.values())
ax.scatter(x_vals, y_vals, c="blue", label="Vertices")

# Label each vertex
for index, (x, y) in vertices.items():
    ax.text(x, y, str(index), fontsize=16, ha="right", va="bottom")

# Set plot limits and labels
ax.set_xlim(0, 5)
ax.set_ylim(0, 5)
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.set_title("Random Vertices within a 10x10 Box")

plt.grid(True)
plt.legend()
plt.show()

In [ ]:
vertices[12] = (np.float64(3.95), np.float64(0.1))
vertices[18] = (np.float64(1.97), np.float64(0.01))
vertices[21] = (np.float64(4.2), np.float64(4.33))
vertices[33] = (np.float64(1.3), np.float64(0.8))
vertices[15] = (np.float64(1.7), np.float64(0.45))
vertices[4] = (np.float64(4.95), np.float64(1))

vertices_x = copy.deepcopy(vertices)
for key in vertices_x:
    vertices_x[key] = (vertices_x[key][0] + 5, vertices_x[key][1])

vertices_y = copy.deepcopy(vertices)
for key in vertices_y:
    vertices_y[key] = (vertices_y[key][0], vertices_y[key][1] + 5)

vertices_xy = copy.deepcopy(vertices)
for key in vertices_xy:
    vertices_xy[key] = (vertices_xy[key][0] + 5, vertices_xy[key][1] + 5)

dictionaries = [vertices, vertices_x, vertices_y, vertices_xy]
combined = {}
count = 0
for d in dictionaries:
    for key in d:
        combined[count] = d[key]
        count += 1

In [ ]:
vertices = combined
# Plotting
fig, ax = plt.subplots(figsize=(10, 10))

# Scatter plot of vertices
x_vals, y_vals = zip(*vertices.values())
ax.scatter(x_vals, y_vals, c="blue", label="Vertices")

# Label each vertex
for index, (x, y) in vertices.items():
    ax.text(x, y, str(index), fontsize=8, ha="right", va="bottom")

# Set plot limits and labels
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.set_title("Random Vertices within a 10x10 Box")

plt.grid(True)
plt.legend()
plt.show()

In [ ]:
positions = np.array([[combined[key][0], combined[key][1]] for key in combined])
# np.savetxt("vertices_big.csv", positions, delimiter=",")

In [ ]:
verts = [
    [25, 7, 11, 34],
    [47, 38, 40, 27, 34, 11],
    [40, 20, 29, 8, 14, 27],
    [32, 37, 35, 29, 20],
    [48, 23, 21, 10, 39, 37, 32],
    [36, 49, 48, 32, 20, 40, 38],
    [49, 46, 22, 23, 48],
    [5, 33, 44, 9, 36, 38, 47],
    [45, 26, 41, 1, 9, 44],
    [41, 3, 42, 28, 1],
    [1, 28, 46, 49, 36, 9],
    [42, 13, 22, 46, 28],
    [12, 31, 4, 19, 3, 41, 26],
    [3, 19, 57, 75, 66, 13, 42],
    [19, 4, 55, 97, 61, 57],
    [4, 31, 74, 65, 83, 55],
    [33, 15, 17, 30, 45, 44],
    [31, 12, 110, 121, 100, 106, 74],
    [26, 45, 30, 139, 110, 12],
    [30, 17, 18, 135, 137, 139],
    [18, 143, 108, 129, 135],
    [17, 15, 24, 156, 143, 18],
    [14, 52, 16, 25, 34, 27],
    [22, 13, 66, 2, 0, 21, 23],
    [2, 64, 58, 93, 6, 0],
]


def copy_x(x):
    if x < 50:
        return x + 50
    elif x < 100:
        return (x + 50) % 100
    elif x < 150:
        return x + 50
    else:
        return ((x + 50) % 100) + 100


verts_x = []
for l in verts:
    to_add = []
    for num in l:
        to_add.append(copy_x(num))
    verts_x.append(to_add)

verts.extend(verts_x)

verts_vextend = []
for l in verts:
    to_add = []
    for num in l:
        to_add.append((num + 100) % 200)
    verts_vextend.append(to_add)

verts.extend(verts_vextend)

In [ ]:
# New
counts = {}
for row in verts:
    for num in row:
        if num not in counts:
            counts[num] = 0
        counts[num] += 1

for key in counts:
    if counts[key] != 3:
        print(key, "\t", counts[key])

In [ ]:
# New
verts = [
    [25, 7, 11, 34],
    [47, 38, 40, 27, 34, 11],
    [40, 20, 29, 8, 14, 27],
    [32, 37, 35, 29, 20],
    [48, 23, 21, 10, 39, 37, 32],
    [36, 49, 48, 32, 20, 40, 38],
    [49, 46, 22, 23, 48],
    [5, 33, 44, 9, 36, 38, 47],
    [45, 26, 41, 1, 9, 44],
    [41, 3, 42, 28, 1],
    [1, 28, 46, 49, 36, 9],
    [42, 13, 22, 46, 28],
    [12, 31, 4, 19, 3, 41, 26],
    [3, 19, 57, 75, 66, 13, 42],
    [19, 4, 55, 97, 61, 57],
    [4, 31, 74, 65, 83, 55],
    [33, 15, 17, 30, 45, 44],
    [31, 12, 110, 121, 100, 106, 74],
    [26, 45, 30, 139, 110, 12],
    [30, 17, 18, 135, 137, 139],
    [18, 143, 108, 129, 135],
    [17, 15, 24, 156, 143, 18],
    [14, 52, 16, 25, 34, 27],
    [22, 13, 66, 2, 0, 21, 23],
    [2, 64, 58, 93, 6, 0],
]

In [ ]:
# Orig
verts = [
    [25, 7, 11, 47, 33, 34],
    [47, 38, 40, 27, 34, 33],
    [40, 20, 29, 8, 14, 27],
    [32, 37, 15, 35, 29, 20],
    [48, 23, 21, 10, 39, 37, 32],
    [36, 49, 48, 32, 20, 40, 38],
    [49, 46, 4, 22, 23, 48],
    [5, 44, 9, 36, 38, 47],
    [45, 26, 41, 1, 9, 44],
    [41, 3, 42, 28, 1],
    [1, 28, 46, 49, 36, 9],
    [42, 13, 22, 4, 46, 28],
    [12, 31, 19, 3, 41, 26],
    [3, 19, 57, 75, 66, 13, 42],
    [19, 55, 97, 61, 57],
    [19, 31, 74, 67, 80, 95, 94, 55],
    [31, 12, 110, 121, 100, 106, 74],
    [26, 45, 30, 139, 110, 12],
    [30, 17, 18, 135, 115, 137, 139],
    [18, 143, 108, 129, 135],
    [17, 24, 156, 143, 18],
    [14, 52, 16, 25, 34, 27],
    [22, 13, 66, 2, 0, 21, 23],
    [2, 64, 58, 93, 6, 0],
]